In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils, size_control, find_depth_neurons
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.plotting import plotting_utils
from v1_depth_map.figure_utils import depth_selectivity, get_session_list
from v1_depth_map.figure_utils import size_control as plt_size_control
from v1_depth_map.figure_utils import common_utils as plt_common_utils

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 10
READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{READ_VERSION}"/"fig_size_control"
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{VERSION}"/"fig_size_control"
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load data
project = "hey2_3d-vision_foodres_20220101"

# Load example session
session_name = "PZAH10.2d_S20230822"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = size_control.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SizeControl",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full.parent/"neurons_df.pickle")

# Load neurons_df of all sessions
session_list = [
    "PZAH10.2d_S20230822", 
    "PZAH10.2f_S20230815", 
    "PZAH10.2f_S20230907"
    ]
flexilims_session = flz.get_flexilims_session(project)
neurons_df_all = plt_common_utils.concatenate_all_neurons_df(flexilims_session=flexilims_session,
                                                             session_list=session_list,
                                                             filename="neurons_df.pickle",
                                                             cols=None,
                                                             read_iscell=True,
                                                             verbose=True)

In [ ]:
select_neurons = (
    (neurons_df_all.iscell == 1)
    & (neurons_df_all.depth_tuning_test_spearmanr_pval_closedloop < 0.05)
    & (neurons_df_all.depth_tuning_test_spearmanr_rval_closedloop > 0.1)
    )
print(f"Number of neurons: {select_neurons.sum()}")
ROIS = [453, 133, 448]
# compute ylims for neurons' depth tuning
ylims_all = np.zeros((len(ROIS), 2, 4, 2))
for iroi, roi in enumerate(ROIS):
    for iparam, param in enumerate(["depth", "size"]):
        mean_dff_arr = find_depth_neurons.average_dff_for_all_trials(
            trials_df=trials_df_example,
            rs_thr=None,
            rs_thr_max=None,
            still_only=False,
            still_time=0,
            frame_rate=15,
            closed_loop=1,
            param="size",
            )[:, :, roi]
        CI_low, CI_high = common_utils.get_bootstrap_ci(mean_dff_arr)
        ylim = [round(np.nanmin(CI_low), 1), round(np.nanmax(CI_high), 1)]
        ylims_all[iroi, iparam, 0, :] = ylim
        
        for isize, size in enumerate([5,10,20]):
            mean_dff_arr = find_depth_neurons.average_dff_for_all_trials(
                trials_df=trials_df_example[trials_df_example["size"] == size],
                rs_thr=None,
                rs_thr_max=None,
                still_only=False,
                still_time=0,
                frame_rate=15,
                closed_loop=1,
                param="size",
                )[:, :, roi]
            CI_low, CI_high = common_utils.get_bootstrap_ci(mean_dff_arr)
            ylim = [np.nanmin(CI_low), np.nanmax(CI_high)]
            ylims_all[iroi, iparam, isize+1, :] = ylim           
            
    

In [ ]:
cm=1/2.54
fig = plt.figure(figsize=(18 * cm, 18 * cm))
fontsize_dict={"title":7, "label": 7, "tick": 5}
# Example cells: depth tuning at different sphere sizes
ROIS = [453, 133, 448] 
ylims_all[0, 0, :, 0] = 0.2

for iroi, roi in enumerate(ROIS):
    for iparam, param in enumerate(["depth"]):
        if iparam == 0:
            ax=fig.add_axes([0.05 + iroi*0.25, 0.7, 0.15, 0.15])
        else:
            ax=fig.add_axes([0.4, 1 - iroi * 0.2, 0.15, 0.15])
        
        depth_tuning_kwargs = dict(
            rs_thr=None,
            plot_fit=True,
            plot_smooth=False,
            linewidth=1.5,
            closed_loop=1,
            fontsize_dict=fontsize_dict,
            markersize=8,
            markeredgecolor='w',
            linecolor="k",
        )
                  
        depth_selectivity.plot_depth_tuning_curve(
            neurons_df=neurons_df_example,
            trials_df=trials_df_example,
            roi=roi,
            param=param,
            use_col=f"{param}_tuning_popt_closedloop",
            min_sigma=0.5,
            ylim=[np.nanmin(ylims_all[iroi, 0, :, 0]), plt_common_utils.ceil(np.nanmax(ylims_all[iroi, 0, :, 1]),1)],
            **depth_tuning_kwargs,
        )
        if iroi != len(ROIS) - 1:
            ax.set_xlabel("")
        
        depth_tuning_kwargs = dict(
            rs_thr=None,
            plot_fit=True,
            plot_smooth=False,
            linewidth=1.5,
            closed_loop=1,
            fontsize_dict=fontsize_dict,
            markersize=8,
            markeredgecolor='w',
        )
        if iparam == 0:
            ax = fig.add_axes([0.05 + iroi*0.25, 0.45, 0.15, 0.15])
            for size, linecolor in zip([5,10,20], ["skyblue", "royalblue", "navy"]):
                depth_selectivity.plot_depth_tuning_curve(
                    neurons_df=neurons_df_example,
                    trials_df=trials_df_example[trials_df_example["size"] == size],
                    roi=roi,
                    param=param,
                    use_col=f"{param}_tuning_popt_size{size}",
                    min_sigma=0.5,
                    ylim=[np.nanmin(ylims_all[iroi, 0, :, 0])-0.02, plt_common_utils.ceil(np.nanmax(ylims_all[iroi, 0, :, 1]),1)],
                    label=f"{size} degrees",
                    linecolor=linecolor,
                    **depth_tuning_kwargs,
                )
            if iroi == 0:
                plt.legend(fontsize=5, frameon=False, handlelength=1)
            if iroi != len(ROIS) - 1:
                ax.set_xlabel("")

# Scatter plot of preferred depths at different visual angles
select_neurons = (
    (neurons_df_all.iscell == 1)
    & (neurons_df_all.depth_tuning_test_spearmanr_pval_closedloop < 0.05)
    & (neurons_df_all.depth_tuning_test_spearmanr_rval_closedloop > 0.1)
    )
plt_size_control.plot_preferred_depths_sizes_scatter(neurons_df=neurons_df_all[select_neurons], 
                                                     sizes=[5,10,20], 
                                                     plot_x=0.05, 
                                                     plot_y=0.1, 
                                                     plot_width=0.22, 
                                                     plot_height=0.22, 
                                                     fontsize_dict=fontsize_dict)


fig.savefig(SAVE_ROOT/"fig_size_tuning.svg", bbox_inches="tight", dpi=300)

## stat

In [ ]:
# calculate pvals for ratio between preferred depth across size pairs
n_boots = 20000
xcol = ["preferred_depth_size5", "preferred_depth_size5", "preferred_depth_size10"]
ycol = ["preferred_depth_size10", "preferred_depth_size20", "preferred_depth_size20"]
np.random.seed(0)
r, distribution = plt_common_utils.hierarchical_bootstrap_stats(neurons_df_all[select_neurons], 
                                               n_boots, 
                                               xcol=xcol, 
                                               resample_cols=["mouse", "session"], 
                                               ycol=ycol, 
                                               correlation=False,
                                               difference=False,
                                               ratio=True)
for i in range(3):
    pval = plt_common_utils.calculate_pval_from_bootstrap(distribution[:,i], value=1)
    print(f"median {r[i]}, pval {pval}")

In [ ]:
# calculate pvals for correlation between preferred depth across size pairs
n_boots = 20000
xcol = ["preferred_depth_size5", "preferred_depth_size5", "preferred_depth_size10"]
ycol = ["preferred_depth_size10", "preferred_depth_size20", "preferred_depth_size20"]
np.random.seed(0)
r, distribution = plt_common_utils.hierarchical_bootstrap_stats(neurons_df_all[select_neurons], 
                                               n_boots, 
                                               xcol=xcol, 
                                               resample_cols=["mouse", "session"], 
                                               ycol=ycol, 
                                               correlation=True,
                                               difference=False,
                                               ratio=False)
for i in range(3):
    pval = plt_common_utils.calculate_pval_from_bootstrap(distribution[:,i], value=1)
    print(f"median {r[i]}, pval {pval}")